Imports and setting up static constants we need later

In [1]:
#!/usr/bin/python
from __future__ import division
import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

#static business for the counters
RGPcounterurls="d0f355e237dda999f3112d94d3c762c7" # TCA
                

#different walls implement counters in different ways
#verticallifeurls=["https://gyms.vertical-life.info/en/the-climbing-works/counter"]                
#Edenrockurls=["https://wallmarket.herokuapp.com/eden-rock-edinburgh/entries"]

#let's have a list of the ones that are in the same place for convenient sorting/iterating purposes
cities={"Sheffield":["Awesome Walls Sheffield","Foundry","Depot Climbing Sheffield"],
        "Leeds":["Big Depot Leeds","Depot Leeds (Pudsey)"],
        "Manchester":["Manchester Climbing Centre","Awesome Walls Stockport","Depot Manchester"],
        "London":["CroyWall","Here Yonder","VauxWall East","HarroWall","RavensWall","VauxWall West","CanaryWall"],
        "Bristol":["The Church","The Mothership"],
        "Glasgow":["The Prop Store","The Newsroom"]
    }

urlstart="https://portal.rockgympro.com/portal/public/"
urlend="/occupancy"

walls={}

functions

In [2]:
class Wall:
    def __init__(self, fname):
        self.fancyname="fname"  #name of wall
        self.identifier="ident" #3-letter identifier
        self.capacity=""        #capacity
        self.lastupdated=""     #last update string
        self.count=""           #the prize
        self.subLabel=""        #because it's there
        self.city=""            #filled only if it's in the list above

    def barcolour(self): #return red, amber, or green
        if self.capacity==0:
            return(0) #dodge div/0 error
        
        fullness=100*int(self.count)/int(self.capacity)
        if fullness < 30:
            return("#238823")
        if fullness > 30 and fullness < 80:
            return("#FFBF00")
        if fullness > 80:
            return("#D2222D")
        return("#000000")

Go to the site and get the data, display it in a sensible format

In [3]:
newwallids={}
geturl=urlstart+RGPcounterurls+urlend
print("getting "+geturl)
    
#visit the page and fetch the data for munging
page = requests.get(geturl).text

#timestamp
today = datetime.now()
timestamp = {"timechecked" : str(today)}

#read that data in
soup=BeautifulSoup(page, 'html.parser')

for opt in soup.find_all('option'):
    print(opt.get('value'), opt.get('text'))

getting https://portal.rockgympro.com/portal/public/d0f355e237dda999f3112d94d3c762c7/occupancy
 None
UNC None
BRI None
GLA None
PST None


In [4]:
#find all the scripts on the page
for dat in soup.find_all('script'):
    #look for the one we are interested in
    #define the regex to only match the bit which is useful
    regex = r"var data(.*)},    };"
    #change the object into a string for searching
    usefulbits = str(dat)
    
    #just pull out the bit we need
    matches = re.search(regex, usefulbits, re.DOTALL | re.IGNORECASE)
    if matches:
        datastr=matches.group(0).replace("var data = ","")
        datastr=datastr.replace("\n", "")
        datastr=datastr.replace("  ", "")
        datastr=datastr.replace("\'", "\"")
        datastr=datastr.replace("},};", "}}") 
        
        jsondata = json.loads(datastr)
        jsondata.update(timestamp)
        
print(jsondata)

{'BRI': {'capacity': 115, 'count': 19, 'subLabel': 'Current climber count', 'lastUpdate': 'Last updated:&nbspnow(3:42 PM)'}, 'UNC': {'capacity': 50, 'count': 5, 'subLabel': 'Current climber count', 'lastUpdate': 'Last updated:&nbsp3 mins ago (3:40 PM)'}, 'GLA': {'capacity': 100, 'count': 0, 'subLabel': 'Current Occupancy', 'lastUpdate': 'Last updated:&nbsp3 mins ago (3:40 PM)'}, 'PST': {'capacity': 80, 'count': 0, 'subLabel': 'Current Occupancy', 'lastUpdate': 'Last updated:&nbsp2 mins ago (3:41 PM)'}, 'timechecked': '2020-08-12 15:43:01.961497'}


In [5]:
print(json.dumps(jsondata, indent=4, sort_keys=True))

{
    "BRI": {
        "capacity": 115,
        "count": 19,
        "lastUpdate": "Last updated:&nbspnow(3:42 PM)",
        "subLabel": "Current climber count"
    },
    "GLA": {
        "capacity": 100,
        "count": 0,
        "lastUpdate": "Last updated:&nbsp3 mins ago (3:40 PM)",
        "subLabel": "Current Occupancy"
    },
    "PST": {
        "capacity": 80,
        "count": 0,
        "lastUpdate": "Last updated:&nbsp2 mins ago (3:41 PM)",
        "subLabel": "Current Occupancy"
    },
    "UNC": {
        "capacity": 50,
        "count": 5,
        "lastUpdate": "Last updated:&nbsp3 mins ago (3:40 PM)",
        "subLabel": "Current climber count"
    },
    "timechecked": "2020-08-12 15:43:01.961497"
}


In [6]:
from google.cloud import bigquery
from google.oauth2 import service_account

client = bigquery.Client()

dataset_id = 'howfullismywall:walldata'
table_id = 'howfullismywall:walldata.howfullismywall'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.autodetect = True

job = client.load_table_from_file(
    jsondata,
    table_ref,
    job_config=job_config,
)

job.result()

print("loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [7]:
!export GOOGLE_APPLICATION_CREDENTIALS=/Users/a344348/Downloads/howfullismywall-284248362156.json

In [8]:
!echo $GOOGLE_APPLICATION_CREDENTIALS